In [ ]:
import logging
from rasa.core.agent import Agent
from rasa.core.interpreter import RasaNLUInterpreter
from rasa.core.policies import MemoizationPolicy, KerasPolicy, FallbackPolicy, TEDPolicy
from rasa.core.training import interactive
from rasa.core.actions import Action
from rasa.core.events import SlotSet
from rasa.utils import train
import random
import spacy
from transformers import BertTokenizer, BertForSequenceClassification
import torch

logging.basicConfig(level=logging.DEBUG)

# Initialize Spacy and BERT models
nlp = spacy.load("en_core_web_md")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Load pre-trained NLU model
interpreter = RasaNLUInterpreter("models/nlu/default/model_20240101")

# Define the custom action class
class ActionProvideCyberInfo(Action):
    def name(self):
        return "action_ask_question"
    
    def run(self, dispatcher, tracker, domain):
        # Extract topic from slot
        topic = tracker.get_slot("topic")
        context = tracker.latest_message.get('text')

        # Advanced response generation
        response = self.get_dynamic_response(topic, context)

        # Use context-aware responses
        dispatcher.utter_message(text=response)
        return []

    def get_dynamic_response(self, topic, context):
        """ Generate context-based response with dynamic content. """
        if topic == "phishing":
            response = self.get_phishing_info(context)
        elif topic == "malware":
            response = self.get_malware_info(context)
        elif topic == "social_engineering":
            response = self.get_social_engineering_info(context)
        else:
            response = "Sorry, I didn't quite understand. Could you please elaborate?"
        return response
    
    def get_phishing_info(self, context):
        # Simulate real-time dynamic response generation
        if "example" in context:
            return "Phishing is when attackers impersonate legitimate organizations to steal personal information. An example is fake emails from a bank asking for login credentials."
        return "Phishing is a type of attack where attackers trick you into giving sensitive information. Be cautious of suspicious emails or websites."

    def get_malware_info(self, context):
        if "example" in context:
            return "Malware includes viruses, ransomware, and spyware. A well-known example is the WannaCry ransomware attack."
        return "Malware is any harmful software designed to damage or exploit systems. Always update your antivirus software and avoid downloading from untrusted sources."
    
    def get_social_engineering_info(self, context):
        if "example" in context:
            return "Social engineering exploits human psychology, like when an attacker impersonates a trusted colleague to gain access to confidential information."
        return "Social engineering attacks manipulate individuals into divulging confidential information. Always verify requests before sharing sensitive data."


# Step 3: BERT-based Intent Classification for Cybersecurity Awareness
def classify_intent_with_bert(text):
    """ Use BERT to classify intents based on the query. """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class

# Step 4: Dynamic Query Handling
def handle_dynamic_query(query):
    """ Handle user queries with advanced NLP techniques using BERT. """
    intent = classify_intent_with_bert(query)
    
    # You can extend this with specific intent handling logic based on the classification
    if intent == 0:  # Assume '0' corresponds to phishing intent
        return "Phishing is a type of cyber attack. It's where attackers impersonate legitimate organizations to steal sensitive information."
    elif intent == 1:  # Assume '1' corresponds to malware intent
        return "Malware is harmful software like viruses or ransomware that can damage your computer."
    else:
        return "Sorry, I didn’t understand that. Could you please ask something related to cybersecurity?"

# Step 5: Train the Model and Set up the Rasa Chatbot
def train_rasa_model():
    """ Train the Rasa chatbot model using advanced configurations. """
    training_data = [
        {"intent": "greet", "examples": ["Hello", "Hi", "Hey"]},
        {"intent": "ask_phishing", "examples": ["What is phishing?", "Tell me about phishing"]},
        {"intent": "ask_malware", "examples": ["What is malware?", "Explain malware"]},
        {"intent": "ask_social_engineering", "examples": ["What is social engineering?", "Tell me about social engineering"]},
    ]

    # Create Rasa NLU data
    nlu_data = rasa.nlu.training_data.load_data(training_data)
    
    # Define policies and configure Rasa
    policies = [MemoizationPolicy(), KerasPolicy(), FallbackPolicy(fallback_action_name="utter_deny", core_fallback_threshold=0.3)]
    
    # Initialize Rasa agent
    agent = Agent("domain.yml", policies=policies)
    
    # Train the agent with data
    agent.train(training_data)
    agent.persist("models/")
    print("Rasa model trained and saved successfully.")
    
    return agent

# Step 6: Interactive Chatbot
def interactive_chat():
    """ Start an interactive conversation with the chatbot. """
    agent = Agent.load("models/")
    interactive.serve_application(agent)

# Step 7: Simulate a Conversation with Context-Aware Responses
def simulate_conversation():
    """ Simulate a conversation where the bot responds based on context. """
    agent = Agent.load("models/")

    # Handle some queries programmatically
    user_queries = [
        "What is phishing?",
        "Tell me about malware.",
        "What is social engineering?",
        "Explain phishing with an example."
    ]
    
    for query in user_queries:
        print(f"User: {query}")
        handle_dynamic_query(query)

# Step 8: Running the Chatbot
if __name__ == "__main__":
    # Train the model if needed
    model = train_rasa_model()
    
    # Simulate conversation
    simulate_conversation()
    
    # Optionally, start interactive chat
    # interactive_chat()